In [4]:
%run -i ./startup.py

In [11]:
%%writefile /dev/notebook/dc_transform.py
# %load /dev/notebook/dc_transform.py
def dc_transform (db = 'ufrn_wait',dc = 'turmas', key_types={}):
    dbc = db+'/'+dc
    dc_data = dc_join.dict[dbc]['data']
    
    for key in dc_data:
        for key_type in key_types:
            print(key, key_type)
            if key.startswith(key_type):
                key_fcn,key_data,key_args = key_types[key_type];
                for arg in key_args:
                    key_fcn_args = { **{ key_data : dc_data[key] } , **arg }
                    dc_data[key] = key_fcn( **key_fcn_args )
                break
    dc_join.dict[dbc]['key_types'] = key_types


Overwriting /dev/notebook/dc_transform.py


In [ ]:
%%writefile /dev/source/dadosbook/functions/dc_join.py
# %load /dev/notebook/dc_join.py
def dc_join(db='ufrn_wait',dc='turmas',num=[],**kwargs):

    import re

    if not hasattr(dc_join,'dict'):
        dc_join.dict = {}
    if not 'key_types' in kwargs:
        key_types = {}
    if not 'ignore_index' in kwargs:
        ignore_index = True

    db_folder_fcn = lambda db,dc : '/data/%s' % (db)
    db_folder     = db_folder_fcn(db,dc)

    db_folders = os.listdir(db_folder)
    dc_folders = [db_folder+'/'+folder for folder in db_folders if re.findall(dc,folder)]
    dc_files = [dc_folder+'/'+dc_file for dc_folder in dc_folders for dc_file in os.listdir(dc_folder)]
    csv_files = [dc_file for dc_file in dc_files if dc_file.endswith('.csv')]

    dc_data = pd.concat([pd.read_csv(csv,sep=';') for csv in csv_files], ignore_index=ignore_index)
    
    dc = re.sub('[^\w]','',dc)
    dbc = db+'/'+dc
    
    dc_join.dict[dbc] = {'folder':dc_folders, 'csv_files': csv_files, 'db':db, 'dc':dc, 'data':dc_data}

    dc_transform(db = db, dc = dc, key_types=key_types)

    return dc_data


In [13]:
%%writefile /dev/notebook/dc_dict_merge.py
# %load /dev/notebook/dc_dict_merge.py
def dc_dict_merge(dc_formed, dc_list, data=[], db='ufrn_wait'):
    if not ( db + '/' + dc_formed ) in dc_join.dict.keys():
        _list_dict = []
        
        _keys_dc = []
        for dc in dc_list:
            key = db + '/' + dc
            if key in dc_join.dict
                _keys_dc.extend( dc_join.dict[key].keys() )
            else:
                print('%s is not in dc_join.dict keys.' % ( key ) )        
        
        _formed_dict = {}
        for key in pd.unique(_keys_dc):
            _formed_dict[key] = []
            for dc in dc_list:    
                _formed_dict.append( dc_join.dict[dc][key] )

        
        if data:
            _cursos_dict['data'] = dc_cursos
        dc_join.dict['ufrn_wait/cursos'] = _cursos_dict

        
    for dc in cursos_dc:
        try:
            dc_join.dict.pop('ufrn_wait/'+dc)
        except:
            print('.')

Overwriting /dev/notebook/dc_dict_merge.py


In [20]:
%%writefile -a ./startup.py
exec(open('./functions/dc_join.py').read())
exec(open('./functions/dc_transform.py').read())
exec(open('./functions/dc_dict_merge.py').read())

Appending to ./startup.py
